In [27]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import re
import time
import random
import tempfile
from tqdm.notebook import tqdm

# Loading the trianing data

In [3]:
raw_train_orig = pd.read_csv('train_raw.csv')
raw_train = raw_train_orig.copy()

C:\Users\padit\AppData\Local\Temp\ipykernel_25828\900728372.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_train_orig = pd.read_csv('train_raw.csv')


# Initial data inspection

In [4]:
raw_train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [5]:
raw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [6]:
raw_train.isna().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [8]:
raw_train.shape

(100000, 28)

In [9]:
raw_train.describe()

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,84998.000000,100000.000000,100000.00000,100000.000000,100000.000000,98035.000000,100000.000000,100000.000000
mean,4194.170850,17.091280,22.47443,72.466040,21.068780,27.754251,32.285173,1403.118217
std,3183.686167,117.404834,129.05741,466.422621,14.860104,193.177339,5.116875,8306.041270
min,303.645417,-1.000000,0.00000,1.000000,-5.000000,0.000000,20.000000,0.000000
25%,1625.568229,3.000000,4.00000,8.000000,10.000000,3.000000,28.052567,30.306660
50%,3093.745000,6.000000,5.00000,13.000000,18.000000,6.000000,32.305784,69.249473
75%,5957.448333,7.000000,7.00000,20.000000,28.000000,9.000000,36.496663,161.224249
max,15204.633333,1798.000000,1499.00000,5797.000000,67.000000,2597.000000,50.000000,82331.000000


# Cleaning the data

## Function to clean objects columns with NaN values or values that are not string types

In [10]:
def text_cleaning(data):
    """
    Function to clean textual data
    Params:
        data: The data to be cleaned
    Returns: 
        data: If the data is not NaN and is an instance of a string type the data is returned without modification
              If the data is not an instance of a string, it is treated as a string and any underscore, commas, and quotation marks are removed

    """
    if data is np.NaN or not isinstance(data, str):
        return data
    else:
        return str(data).strip('_ ,"')

In [11]:
clean_df = raw_train.applymap(text_cleaning).replace(['', 'nan', '!@9#%8', '#F%$D@*&8'], np.NaN)

C:\Users\padit\AppData\Local\Temp\ipykernel_25828\1474753463.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clean_df = raw_train.applymap(text_cleaning).replace(['', 'nan', '!@9#%8', '#F%$D@*&8'], np.NaN)


In [12]:
clean_df.select_dtypes('O').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   ID                       100000 non-null  object
 1   Customer_ID              100000 non-null  object
 2   Month                    100000 non-null  object
 3   Name                     90015 non-null   object
 4   Age                      100000 non-null  object
 5   SSN                      94428 non-null   object
 6   Occupation               92938 non-null   object
 7   Annual_Income            100000 non-null  object
 8   Num_of_Loan              100000 non-null  object
 9   Type_of_Loan             88592 non-null   object
 10  Num_of_Delayed_Payment   92998 non-null   object
 11  Changed_Credit_Limit     97909 non-null   object
 12  Credit_Mix               79805 non-null   object
 13  Outstanding_Debt         100000 non-null  object
 14  Credit_History_Age   

## Converting the columns to appropriate types

In [13]:
clean_df['ID'] = clean_df.ID.apply(lambda x: int(x, 16))
clean_df['Customer_ID'] = clean_df.Customer_ID.apply(lambda x: int(x[4:], 16))
clean_df['Month'] = pd.to_datetime(clean_df.Month, format='%B').dt.month
clean_df['Age'] = clean_df.Age.astype(int) 
clean_df['SSN'] = clean_df.SSN.apply(lambda x: x if x is np.NaN else int(str(x).replace('-', ''))).astype(float)
clean_df['Annual_Income'] = clean_df.Annual_Income.astype(float)
clean_df['Num_of_Loan'] = clean_df.Num_of_Loan.astype(int) 
clean_df['Num_of_Delayed_Payment'] = clean_df.Num_of_Delayed_Payment.astype(float)
clean_df['Changed_Credit_Limit'] = clean_df.Changed_Credit_Limit.astype(float)
clean_df['Outstanding_Debt'] = clean_df.Outstanding_Debt.astype(float)
clean_df['Amount_invested_monthly'] = clean_df.Amount_invested_monthly.astype(float)
clean_df['Monthly_Balance'] = clean_df.Monthly_Balance.astype(float)

## Fixing credit history month column so it represents the time period as a total count of months

In [16]:
def convert_month(x):
    """
    Function to convert month column to a count of months
    Params:
        x: Input data
    Returns:
        x: Cleaned data
    """
    if pd.notnull(x):
        num1 = int(x.split(' ')[0])
        num2 = int(x.split(' ')[3])
      
        return (num1*12)+num2
    else:
        return x

In [17]:
clean_df['Credit_History_Age'] = clean_df.Credit_History_Age.apply(lambda x: convert_month(x)).astype(float)

## Converting loan type to lower case and removing 'and' to standardise the strucure of data in this column

In [18]:
clean_df['Type_of_Loan']

0        Auto Loan, Credit-Builder Loan, Personal Loan,...
1        Auto Loan, Credit-Builder Loan, Personal Loan,...
2        Auto Loan, Credit-Builder Loan, Personal Loan,...
3        Auto Loan, Credit-Builder Loan, Personal Loan,...
4        Auto Loan, Credit-Builder Loan, Personal Loan,...
                               ...                        
99995                          Auto Loan, and Student Loan
99996                          Auto Loan, and Student Loan
99997                          Auto Loan, and Student Loan
99998                          Auto Loan, and Student Loan
99999                          Auto Loan, and Student Loan
Name: Type_of_Loan, Length: 100000, dtype: object

In [19]:
def clean_loan_type_text(text):
    """
    Function to clean the rows in the loan type column to standardise them
    
    Parameters:
        text: The text to be cleaned
    Returns:
        test: The cleaned text
    """
    if pd.notna(text):
        text = text.lower()
        text = text.replace('and ', '')
        text = text.replace(', ', ',')
        text = text.strip()
    return text


In [20]:
clean_df['Type_of_Loan'] = clean_df['Type_of_Loan'].apply(clean_loan_type_text)

In [21]:
clean_df['Type_of_Loan']

0        auto loan,credit-builder loan,personal loan,ho...
1        auto loan,credit-builder loan,personal loan,ho...
2        auto loan,credit-builder loan,personal loan,ho...
3        auto loan,credit-builder loan,personal loan,ho...
4        auto loan,credit-builder loan,personal loan,ho...
                               ...                        
99995                               auto loan,student loan
99996                               auto loan,student loan
99997                               auto loan,student loan
99998                               auto loan,student loan
99999                               auto loan,student loan
Name: Type_of_Loan, Length: 100000, dtype: object

## Filling none and NaN values with the mode of each categorical column

In [26]:
def fill_na_with_self_mode(df):
    """
    Function to fill none and NaN values with the mode for categorical columns
    Params:
        df: The dataframe
    Returns:
        df: The dataframe with none and NaN values replaced with the mode for categorical variables
    """
    for col in df.columns:
        if df[col].dtype == 'object': 
            if df[col].isnull().any() or (df[col] == 'None').any():
                mode_value = df[col].mode().dropna().iloc[0] if not df[col].mode().dropna().empty else None
                df[col] = df[col].fillna(mode_value)
                df[col] = df[col].replace('None', mode_value)

    return df

In [24]:
clean_df = fill_na_with_self_mode(clean_df)

In [25]:
clean_df.isna().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                            0
Age                             0
SSN                          5572
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                    0
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit         2091
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

# Cleaning numerical columns yb removing outliers and filling in NaN values with the mode

In [30]:
def clean_numerical_data(df, groupby=None, replace_with="mode", iqr_multiplier=1.5):
    """
    Function to detect outliers and clean the numericla columns
    Params:
        df: The dataset
        groupby: Column to group by
        replace_with: Statistical method to use to replace incorrect values
        iqr_multiplier: The interquartile range multiplier
    Returns:
        df: The cleaned dataset
    """
    
    for column in df.select_dtypes(include=['number']).columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (iqr * iqr_multiplier)
        upper_bound = q3 + (iqr * iqr_multiplier)

        # Detect outliers
        outliers = (df[column] < lower_bound) | (df[column] > upper_bound)
        print(f"Outliers detected in {column}: {df[column][outliers].count()} instances")

        if groupby and replace_with == "mode":
            mode_by_group = df.groupby(groupby)[column].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
            df[column] = np.where(outliers, mode_by_group, df[column])
        elif replace_with == "median":
            median_by_group = df.groupby(groupby)[column].transform('median')
            df[column] = np.where(outliers, median_by_group, df[column])
        elif replace_with == "mean":
            mean_by_group = df.groupby(groupby)[column].transform('mean')
            df[column] = np.where(outliers, mean_by_group, df[column])
        else:
            df[column][outliers] = np.nan
    
    return df

In [31]:
clean_df = clean_numerical_data(clean_df, groupby = 'Customer_ID', replace_with="mode", iqr_multiplier=1.5)

Outliers detected in ID: 0 instances
Outliers detected in Customer_ID: 0 instances
Outliers detected in Month: 0 instances
Outliers detected in Age: 2781 instances
Outliers detected in SSN: 0 instances
Outliers detected in Annual_Income: 2783 instances
Outliers detected in Monthly_Inhand_Salary: 1683 instances
Outliers detected in Num_Bank_Accounts: 1315 instances
Outliers detected in Num_Credit_Card: 2271 instances
Outliers detected in Interest_Rate: 2034 instances
Outliers detected in Num_of_Loan: 4348 instances
Outliers detected in Delay_from_due_date: 4002 instances
Outliers detected in Num_of_Delayed_Payment: 736 instances
Outliers detected in Changed_Credit_Limit: 668 instances
Outliers detected in Num_Credit_Inquiries: 1650 instances
Outliers detected in Outstanding_Debt: 5272 instances
Outliers detected in Credit_Utilization_Ratio: 4 instances
Outliers detected in Credit_History_Age: 0 instances
Outliers detected in Total_EMI_per_month: 6795 instances
Outliers detected in Amoun